In [367]:
# Python
import pandas as pd
import numpy as np
#
from pathlib import Path
#
import xgboost
#
import importlib
import utilities.train_test as train_test
import utilities.comparison as comparison

In [368]:
df = pd.read_csv('../../../data/df_monthly_returns_complete_percentage.csv', index_col='Date')
df_overview = pd.read_csv('../../../data/df_overview.csv', index_col=0)

In [448]:
file = Path("../../../data/df_tabular.csv")

if file.exists():
    df_tabular = pd.read_csv(file, index_col=0)
else:
    importlib.reload(train_test)
    df_tabular = train_test.get_dataframe_tabular(df)
    df_tabular.to_csv(file)

In [449]:
df_tabular

,month,year,date,m_return(t-11),m_return(t-10),m_return(t-9),m_return(t-8),m_return(t-7),m_return(t-6),m_return(t-5),...,m_return_target(t+4),m_return_target(t+5),m_return_target(t+6),m_return_target(t+7),m_return_target(t+8),m_return_target(t+9),m_return_target(t+10),m_return_target(t+11),m_return_target(t+12),stock_ticker_label
0,10,2000,2000-10-01,1.13,1.11,1.02,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.01,1.0,1.00,1.00,1371
1,11,2000,2000-11-01,1.11,1.02,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.01,1.00,1.0,1.00,1.00,1371
2,12,2000,2000-12-01,1.02,1.00,1.00,1.00,1.00,1.00,1.01,...,1.00,1.00,1.00,1.01,1.00,1.00,1.0,1.00,1.00,1371
3,1,2001,2001-01-01,1.00,1.00,1.00,1.00,1.00,1.01,1.00,...,1.00,1.00,1.01,1.00,1.00,1.00,1.0,1.00,1.01,1371
4,2,2001,2001-02-01,1.00,1.00,1.00,1.00,1.01,1.00,1.00,...,1.00,1.01,1.00,1.00,1.00,1.00,1.0,1.01,1.00,1371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474406,4,2024,2024-04-01,0.98,1.07,1.13,0.95,0.93,1.02,1.09,...,1.04,0.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1549
474407,5,2024,2024-05-01,1.07,1.13,0.95,0.93,1.02,1.09,1.09,...,0.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1549
474408,6,2024,2024-06-01,1.13,0.95,0.93,1.02,1.09,1.09,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1549
474409,7,2024,2024-07-01,0.95,0.93,1.02,1.09,1.09,1.00,1.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1549


### Missing values

In [450]:
df_tabular[df_tabular["m_return_target(t+1)"].isna()][["date", "stock_ticker_label"]]


,date,stock_ticker_label


## Direct forecasting

## 1 Month

In [451]:
importlib.reload(train_test)

X_train, y_train, X_test, y_test = train_test.get_train_test(df_tabular, months=60)

model = xgboost.XGBRegressor(n_estimators=1000)
model.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_test, y_test)],
          verbose=False)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [479]:
importlib.reload(comparison)
months_1m = 1
X_train_1m, y_train_1m, X_test_1m, y_test_1m = train_test.get_train_test(df_tabular, months=1)
# reset index
X_train_1m.reset_index(drop=True, inplace=True)
y_train_1m.reset_index(drop=True, inplace=True)
X_test_1m.reset_index(drop=True, inplace=True)
y_test_1m.reset_index(drop=True, inplace=True)

In [481]:
# Predictions
y_train_pred_1m = model.predict(X_train_1m)
y_test_pred_1m = model.predict(X_test_1m)

In [482]:
importlib.reload(comparison)
y_train_mean_pred_1m, y_test_mean_pred_1m = comparison.get_train_test_mean_pred(y_train_pred_1m, y_test_pred_1m, len(df.columns))

### 1 month Actual vs Prediction

In [483]:
importlib.reload(comparison)
comparison.generate_plot(df, df_tabular, y_train_mean_pred_1m, y_test_mean_pred_1m)

#### Overview table

In [613]:
importlib.reload(comparison)

# Get train true values followed with predicted month/s
y_train_1m_list = y_train_1m['m_return_target(t+1)'].tolist()
y_test_1m_list = y_test_pred_1m.tolist()
#
df_to_evaluate_1m = comparison.get_df_to_evaluate(df, y_train_1m_list, y_test_1m_list)
df_to_evaluate_1m = df_to_evaluate_1m - 1
df_to_evaluate_1m

,RS1.L,KE,TEG.DE,LEG.DE,SCS,HNI,AVT,ACCO,VNA.DE,7912.T,...,DEQ.DE,KIDS,HALO,MATW,9842.T,KVHI,MOON.L,NEO,6055.T,UNP
0,0.000000,0.00000,-0.100000,0.000000,-0.230000,0.010000,-0.340000,0.000000,0.000000,-0.06000,...,0.000000,0.000000,0.000000,0.070000,0.000000,-0.340000,0.000000,0.00000,0.000000,-0.010000
1,0.000000,0.00000,0.000000,-0.060000,0.030000,0.050000,0.220000,0.000000,-0.040000,0.05000,...,0.000000,0.000000,0.000000,0.090000,0.000000,-0.160000,0.000000,0.00000,0.000000,0.090000
2,0.010000,0.00000,0.550000,0.040000,0.090000,-0.020000,0.280000,-0.080000,0.020000,-0.06000,...,0.000000,0.290000,-0.090000,-0.020000,-0.040000,0.690000,-0.140000,-0.24000,0.000000,0.050000
3,0.000000,-0.03000,0.090000,-0.130000,-0.050000,-0.010000,-0.110000,-0.020000,-0.100000,-0.08000,...,-0.480000,0.000000,0.000000,0.010000,0.000000,-0.260000,0.000000,0.00000,0.000000,0.040000
4,0.000000,0.00000,-0.060000,0.000000,-0.150000,-0.070000,-0.160000,0.000000,0.000000,0.02000,...,1.180000,0.000000,0.000000,0.050000,0.000000,0.150000,0.000000,0.00000,0.000000,0.020000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,-0.050000,0.10000,0.060000,0.020000,0.140000,0.120000,0.120000,0.060000,0.060000,0.06000,...,0.090000,0.070000,0.160000,0.050000,-0.010000,0.080000,0.030000,-0.02000,-0.140000,-0.020000
283,0.000000,-0.04000,-0.040000,-0.030000,-0.050000,-0.040000,-0.060000,-0.060000,-0.050000,0.11000,...,0.070000,-0.090000,0.180000,-0.110000,0.010000,-0.100000,0.190000,0.01000,0.140000,-0.020000
284,0.190000,0.08000,0.020000,0.060000,0.120000,0.220000,0.050000,0.090000,0.070000,-0.08000,...,0.100000,0.070000,0.060000,0.160000,-0.060000,-0.040000,0.140000,0.28000,-0.100000,0.090000
285,-0.040000,-0.22000,0.060000,0.080000,-0.020000,-0.020000,0.030000,0.070000,0.100000,0.06000,...,-0.050000,0.040000,0.160000,-0.130000,-0.040000,0.000000,-0.030000,-0.07000,-0.110000,0.040000


In [666]:
importlib.reload(comparison)
df_pred_1m, raw_weights_1m, mu_1m, S_1m, sigma_1m, sharpe_1m = comparison.get_portfolio_performance(df_to_evaluate_1m,
                                                                                                    "xgbooster_weights_1m.csv",
                                                                                                    min_avg_return=-0.5,
                                                                                                    months=12)

allocation, leftover = comparison.create_discrete_allocation(df_pred_1m, raw_weights_1m)

# Create overview
df_view_1m = pd.DataFrame.from_dict(raw_weights_1m, orient='index', columns=['max_sharpe_weight'])
# Extract volatilities (square root of diagonal elements)
df_view_1m['avg_annual_volatility'] = pd.Series(np.sqrt(np.diag(S_1m)), index=S_1m.columns).values
# Set annual returns
df_view_1m['avg_annual_return'] = mu_1m.values
#
df_view_1m

Expected annual return: 176.7%
Annual volatility: 68.5%
Sharpe Ratio: 2.55
Discrete allocation: {'RS1.L': 5, 'KE': 6, 'TEG.DE': 3, 'LEG.DE': 4, 'SCS': 4, 'HNI': 4, 'AVT': 6, 'ACCO': 10, 'VNA.DE': 3, '7912.T': 5, 'KEYS': 4, 'SGRO.L': 5, 'CBRE': 4, 'BRC': 5, 'PGRE': 4, 'PSON.L': 701, 'BBOX.L': 4, 'REL.L': 314, 'TMV.DE': 20, 'BHE': 1165, 'PLXS': 9, 'GPE.L': 26, 'HAS': 3, 'CHGG': 11, 'JLL': 5, 'LAND.L': 5, 'OLED': 7, 'CDW': 15, 'FLEX': 8, 'REZI': 4, 'TPL': 3, 'ARW': 4, 'NOVT': 3, 'RYN': 3, 'SNX': 3, '7911.T': 6, 'CMPR': 10, 'DLAR.L': 6, '2379.T': 1, '6754.T': 1, 'SHA.DE': 14, 'TTMI': 3, '3234.T': 2, 'PBI': 182, 'BYG.L': 5, 'ASGN': 11, 'COA.L': 6, 'VVV': 3, 'PAGE.L': 6, 'DB1.DE': 4, 'AVB': 7, '3309.T': 2, 'STWD': 4, 'GROW.L': 1, 'MTO.L': 10, 'KFY': 4, 'BLND.L': 3, 'RGLD': 4, 'KFRC': 5, 'IWG.L': 8, '8954.T': 11, 'ZIL2.DE': 2, 'SANM': 329, 'SXS.L': 5, 'YETI': 10, 'ACN': 4, 'HPP': 15, '7893.T': 9, 'SVS.L': 11, 'AGNC': 12, 'CTS': 9, 'IPG': 4, 'BMI': 5, 'FUTR.L': 8, 'EBOX.L': 3, 'SCSC': 6, 'MAN'

,max_sharpe_weight,avg_annual_volatility,avg_annual_return
RS1.L,0.0,3.485498,0.614151
KE,0.0,3.474583,-0.043396
TEG.DE,0.0,3.701630,1.597632
LEG.DE,0.0,3.546676,1.147623
SCS,0.0,3.621287,1.171782
...,...,...,...
KVHI,0.0,3.334557,0.074981
MOON.L,0.0,3.358550,0.617740
NEO,0.0,3.619571,1.075490
6055.T,0.0,4.581623,0.835447


## 6 Months

In [668]:
importlib.reload(train_test)
X_train, y_train, X_test, y_test = train_test.get_train_test(df_tabular, months=60, target_key='m_return_target(t+6)')

# 
model = xgboost.XGBRegressor(n_estimators=1000)
model.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_test, y_test)],
          verbose=False)

In [669]:
months_6m = 6
X_train_6m, y_train_6m, X_test_6m, y_test_6m = train_test.get_train_test(df_tabular, months=6,
                                                                         target_key='m_return_target(t+6)')

# Predictions
y_train_pred_6m = model.predict(X_train_6m)
y_test_pred_6m = model.predict(X_test_6m)

#### Train-Data

In [671]:
importlib.reload(comparison)
y_train_mean_pred_6m, y_test_mean_pred_6m = comparison.get_train_test_mean_pred(y_train_pred_6m, y_test_pred_6m, len(df.columns))

### 6 Months Actual vs Prediction

In [672]:
importlib.reload(comparison)
comparison.generate_plot(df, df_tabular, y_train_mean_pred_6m, y_test_mean_pred_6m)

#### Overview table

In [675]:
# Get train true values followed with predicted month/s
y_train_6m_list = y_train_6m['m_return_target(t+6)'].tolist()
y_test_6m_list = y_test_pred_6m.tolist()
#
df_to_evaluate_6m = comparison.get_df_to_evaluate(df, y_train_1m_list, y_test_1m_list)
df_to_evaluate_6m = df_to_evaluate_6m - 1
df_to_evaluate_6m

,RS1.L,KE,TEG.DE,LEG.DE,SCS,HNI,AVT,ACCO,VNA.DE,7912.T,...,DEQ.DE,KIDS,HALO,MATW,9842.T,KVHI,MOON.L,NEO,6055.T,UNP
0,0.000000,0.00000,-0.100000,0.000000,-0.230000,0.010000,-0.340000,0.000000,0.000000,-0.06000,...,0.000000,0.000000,0.000000,0.070000,0.000000,-0.340000,0.000000,0.00000,0.000000,-0.010000
1,0.000000,0.00000,0.000000,-0.060000,0.030000,0.050000,0.220000,0.000000,-0.040000,0.05000,...,0.000000,0.000000,0.000000,0.090000,0.000000,-0.160000,0.000000,0.00000,0.000000,0.090000
2,0.010000,0.00000,0.550000,0.040000,0.090000,-0.020000,0.280000,-0.080000,0.020000,-0.06000,...,0.000000,0.290000,-0.090000,-0.020000,-0.040000,0.690000,-0.140000,-0.24000,0.000000,0.050000
3,0.000000,-0.03000,0.090000,-0.130000,-0.050000,-0.010000,-0.110000,-0.020000,-0.100000,-0.08000,...,-0.480000,0.000000,0.000000,0.010000,0.000000,-0.260000,0.000000,0.00000,0.000000,0.040000
4,0.000000,0.00000,-0.060000,0.000000,-0.150000,-0.070000,-0.160000,0.000000,0.000000,0.02000,...,1.180000,0.000000,0.000000,0.050000,0.000000,0.150000,0.000000,0.00000,0.000000,0.020000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,-0.050000,0.10000,0.060000,0.020000,0.140000,0.120000,0.120000,0.060000,0.060000,0.06000,...,0.090000,0.070000,0.160000,0.050000,-0.010000,0.080000,0.030000,-0.02000,-0.140000,-0.020000
283,0.000000,-0.04000,-0.040000,-0.030000,-0.050000,-0.040000,-0.060000,-0.060000,-0.050000,0.11000,...,0.070000,-0.090000,0.180000,-0.110000,0.010000,-0.100000,0.190000,0.01000,0.140000,-0.020000
284,0.190000,0.08000,0.020000,0.060000,0.120000,0.220000,0.050000,0.090000,0.070000,-0.08000,...,0.100000,0.070000,0.060000,0.160000,-0.060000,-0.040000,0.140000,0.28000,-0.100000,0.090000
285,-0.040000,-0.22000,0.060000,0.080000,-0.020000,-0.020000,0.030000,0.070000,0.100000,0.06000,...,-0.050000,0.040000,0.160000,-0.130000,-0.040000,0.000000,-0.030000,-0.07000,-0.110000,0.040000


In [677]:
importlib.reload(comparison)
df_pred_6m, raw_weights_6m, mu_6m, S_6m, sigma_6m, sharpe_6m = comparison.get_portfolio_performance(df_to_evaluate_6m,
                                                                                                    "xgbooster_weights_6m.csv",
                                                                                                    min_avg_return=-0.5,
                                                                                                    months=12)
comparison.create_discrete_allocation(df_pred_6m, raw_weights_6m)

# Create overview
df_view_6m = pd.DataFrame.from_dict(raw_weights_6m, orient='index', columns=['max_sharpe_weight'])
# Extract volatilities (square root of diagonal elements)
df_view_6m['avg_annual_volatility'] = pd.Series(np.sqrt(np.diag(S_6m)), index=S_6m.columns).values
# Set annual returns
df_view_6m['avg_annual_return'] = mu_6m.values
#
df_view_6m

Expected annual return: 176.7%
Annual volatility: 68.5%
Sharpe Ratio: 2.55
Discrete allocation: {'RS1.L': 5, 'KE': 6, 'TEG.DE': 3, 'LEG.DE': 4, 'SCS': 4, 'HNI': 4, 'AVT': 6, 'ACCO': 10, 'VNA.DE': 3, '7912.T': 5, 'KEYS': 4, 'SGRO.L': 5, 'CBRE': 4, 'BRC': 5, 'PGRE': 4, 'PSON.L': 701, 'BBOX.L': 4, 'REL.L': 314, 'TMV.DE': 20, 'BHE': 1165, 'PLXS': 9, 'GPE.L': 26, 'HAS': 3, 'CHGG': 11, 'JLL': 5, 'LAND.L': 5, 'OLED': 7, 'CDW': 15, 'FLEX': 8, 'REZI': 4, 'TPL': 3, 'ARW': 4, 'NOVT': 3, 'RYN': 3, 'SNX': 3, '7911.T': 6, 'CMPR': 10, 'DLAR.L': 6, '2379.T': 1, '6754.T': 1, 'SHA.DE': 14, 'TTMI': 3, '3234.T': 2, 'PBI': 182, 'BYG.L': 5, 'ASGN': 11, 'COA.L': 6, 'VVV': 3, 'PAGE.L': 6, 'DB1.DE': 4, 'AVB': 7, '3309.T': 2, 'STWD': 4, 'GROW.L': 1, 'MTO.L': 10, 'KFY': 4, 'BLND.L': 3, 'RGLD': 4, 'KFRC': 5, 'IWG.L': 8, '8954.T': 11, 'ZIL2.DE': 2, 'SANM': 329, 'SXS.L': 5, 'YETI': 10, 'ACN': 4, 'HPP': 15, '7893.T': 9, 'SVS.L': 11, 'AGNC': 12, 'CTS': 9, 'IPG': 4, 'BMI': 5, 'FUTR.L': 8, 'EBOX.L': 3, 'SCSC': 6, 'MAN'

,max_sharpe_weight,avg_annual_volatility,avg_annual_return
RS1.L,0.0,3.485498,0.614151
KE,0.0,3.474583,-0.043396
TEG.DE,0.0,3.701630,1.597632
LEG.DE,0.0,3.546676,1.147623
SCS,0.0,3.621287,1.171782
...,...,...,...
KVHI,0.0,3.334557,0.074981
MOON.L,0.0,3.358550,0.617740
NEO,0.0,3.619571,1.075490
6055.T,0.0,4.581623,0.835447


## 12 Months 

In [ ]:
importlib.reload(train_test)
X_train, y_train, X_test, y_test = train_test.get_train_test(df_tabular, months=60, target_key='m_return_target(t+12)')

# 
model = xgboost.XGBRegressor(n_estimators=1000)
model.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_test, y_test)],
          verbose=False)

In [678]:
months_12m = 12
X_train_12m, y_train_12m, X_test_12m, y_test_12m = train_test.get_train_test(df_tabular, months=12,
                                                                             target_key='m_return_target(t+12)')

In [679]:
# Predictions
y_train_pred_12m = model.predict(X_train_12m)
y_test_pred_12m = model.predict(X_test_12m)

In [680]:
X_test_12m

,month,year,m_return(t-11),m_return(t-10),m_return(t-9),m_return(t-8),m_return(t-7),m_return(t-6),m_return(t-5),m_return(t-4),...,m_return(t-2),m_return(t-1),m_return(t),m_return_target(t+2),m_return_target(t+4),m_return_target(t+6),m_return_target(t+8),m_return_target(t+10),m_return_target(t+12),stock_ticker_label
275,9,2023,0.99,0.96,0.98,1.05,1.04,0.94,1.01,0.86,...,1.05,0.97,0.97,1.10,0.96,0.96,0.95,1.19,0.99,1371
276,10,2023,0.96,0.98,1.05,1.04,0.94,1.01,0.86,0.95,...,0.97,0.97,0.92,1.11,0.96,1.01,1.00,0.96,NaN,1371
277,11,2023,0.98,1.05,1.04,0.94,1.01,0.86,0.95,1.05,...,0.97,0.92,1.10,0.96,0.96,0.95,1.19,0.99,NaN,1371
278,12,2023,1.05,1.04,0.94,1.01,0.86,0.95,1.05,0.97,...,0.92,1.10,1.11,0.96,1.01,1.00,0.96,NaN,NaN,1371
279,1,2024,1.04,0.94,1.01,0.86,0.95,1.05,0.97,0.97,...,1.10,1.11,0.96,0.96,0.95,1.19,0.99,NaN,NaN,1371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474406,4,2024,0.98,1.07,1.13,0.95,0.93,1.02,1.09,1.09,...,1.04,0.97,0.96,0.98,1.04,NaN,NaN,NaN,NaN,1549
474407,5,2024,1.07,1.13,0.95,0.93,1.02,1.09,1.09,1.00,...,0.97,0.96,0.98,1.09,0.99,NaN,NaN,NaN,NaN,1549
474408,6,2024,1.13,0.95,0.93,1.02,1.09,1.09,1.00,1.04,...,0.96,0.98,0.98,1.04,NaN,NaN,NaN,NaN,NaN,1549
474409,7,2024,0.95,0.93,1.02,1.09,1.09,1.00,1.04,0.97,...,0.98,0.98,1.09,0.99,NaN,NaN,NaN,NaN,NaN,1549


#### Train-Data

In [683]:
importlib.reload(comparison)
y_train_mean_pred_12m, y_test_mean_pred_12m = comparison.get_train_test_mean_pred(y_train_pred_12m, y_test_pred_12m, len(df.columns))

### 12 Months Actual vs Prediction

In [684]:
importlib.reload(comparison)
comparison.generate_plot(df, df_tabular, y_train_mean_pred_12m, y_test_mean_pred_12m)

#### Overview table

In [690]:
importlib.reload(comparison)

# Get train true values followed with predicted month/s
y_train_12m_list = y_train_12m['m_return_target(t+12)'].tolist()
y_test_12m_list = y_test_pred_12m.tolist()
#
df_to_evaluate_12m = comparison.get_df_to_evaluate(df, y_train_12m_list, y_test_12m_list)
df_to_evaluate_12m = df_to_evaluate_12m - 1
df_to_evaluate_12m

,RS1.L,KE,TEG.DE,LEG.DE,SCS,HNI,AVT,ACCO,VNA.DE,7912.T,...,DEQ.DE,KIDS,HALO,MATW,9842.T,KVHI,MOON.L,NEO,6055.T,UNP
0,0.000000,0.000000,0.320000,0.000000,0.050000,0.100000,0.140000,0.000000,0.000000,0.140000,...,0.000000,0.000000,0.000000,0.040000,0.000000,0.530000,0.000000,0.000000,0.010000,0.110000
1,0.000000,0.000000,0.050000,-0.020000,0.090000,0.060000,0.150000,1.430000,-0.020000,0.010000,...,0.000000,0.000000,0.000000,0.050000,0.000000,-0.170000,0.000000,0.000000,0.000000,0.060000
2,0.000000,0.000000,-0.150000,0.000000,0.040000,0.090000,0.070000,0.000000,0.000000,-0.010000,...,-0.120000,0.000000,0.000000,0.020000,0.000000,-0.010000,0.000000,0.000000,0.000000,0.040000
3,0.010000,0.000000,0.040000,0.000000,0.070000,0.010000,0.050000,-0.650000,0.000000,-0.100000,...,0.000000,0.000000,0.000000,0.010000,0.000000,0.230000,0.000000,-0.050000,-0.010000,0.090000
4,0.000000,0.000000,-0.020000,0.000000,-0.030000,-0.030000,-0.010000,0.060000,0.000000,0.100000,...,0.010000,-0.100000,0.000000,-0.030000,0.000000,0.020000,0.000000,0.000000,0.010000,-0.020000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,0.244577,0.156353,0.366744,0.682462,0.407572,0.722693,0.135381,0.099747,0.376074,0.294111,...,0.314119,-0.048038,0.410128,0.023295,0.430739,0.187370,0.253476,0.426754,0.726424,0.372873
283,0.783048,0.260594,0.327395,1.077794,0.837635,0.516154,0.407484,0.819195,0.779750,0.498811,...,0.859540,0.192757,0.303528,0.189406,0.420859,0.268963,0.168814,0.298489,0.828836,0.278691
284,-0.022296,-0.156392,0.255224,0.461186,0.081146,0.074576,0.077683,0.374535,0.457722,0.368885,...,0.047518,-0.011545,0.297595,-0.072121,0.424665,0.426920,0.310471,0.009284,1.368559,0.197654
285,0.699889,0.047668,0.089610,0.105799,0.107854,0.542740,0.139710,0.052103,0.103307,0.880507,...,0.389076,0.231679,0.147094,-0.003247,0.530417,-0.146416,0.251774,0.124364,1.360146,0.662187


In [695]:
importlib.reload(comparison)
df_pred_12m, raw_weights_12m, mu_12m, S_12m, sigma_12m, sharpe_12m = comparison.get_portfolio_performance(df_to_evaluate_12m,
                                                                                                          "xgbooster_weights_12m.csv",
                                                                                                          min_avg_return=-0.5,
                                                                                                          months=12)
comparison.create_discrete_allocation(df_pred_12m, raw_weights_12m)

# Create overview
df_view_12m = pd.DataFrame.from_dict(raw_weights_12m, orient='index', columns=['max_sharpe_weight'])
# Extract volatilities (square root of diagonal elements)
df_view_12m['avg_annual_volatility'] = pd.Series(np.sqrt(np.diag(S_12m)), index=S_12m.columns).values
# Set annual returns
df_view_12m['avg_annual_return'] = mu_12m.values
# 
df_view_12m['return_last_period(12m)'] = round(df.tail(12).prod() - 1, 2)
df_view_12m

[-0.04358553886413574, 0.41664886474609375, 0.5675022602081299, 1.1773312091827393, 0.5249886512756348, 0.3171074390411377, 0.3990079164505005, 0.15635335445404053, 0.26059412956237793, -0.15639227628707886, 0.04766845703125, 0.40434956550598145]
mu 22.510101099514678
Expected annual return: 689083.3%
Annual volatility: 521.3%
Sharpe Ratio: 1321.79
Discrete allocation: {'RS1.L': 1, 'KE': 2, 'TEG.DE': 1, 'LEG.DE': 1, 'SCS': 1, 'HNI': 1, 'AVT': 2, 'ACCO': 3, 'VNA.DE': 1, '7912.T': 1, 'KEYS': 1, 'SGRO.L': 1, 'CBRE': 1, 'BRC': 2, 'PGRE': 1, 'PSON.L': 11, 'BBOX.L': 1, 'REL.L': 11, 'TMV.DE': 6, 'BHE': 10, 'PLXS': 3, 'GPE.L': 23, 'HAS': 1, 'JLL': 2, 'LAND.L': 1, 'OLED': 2, 'CDW': 7, 'FLEX': 3, 'REZI': 1, 'TPL': 1, 'ARW': 1, 'NOVT': 1, 'RYN': 1, 'SNX': 1, '7911.T': 2, 'CMPR': 3, 'DLAR.L': 2, 'SHA.DE': 7, 'TTMI': 1, 'PBI': 1, 'BYG.L': 1, 'ASGN': 4, 'COA.L': 2, 'VVV': 1, 'PAGE.L': 2, 'DB1.DE': 1, 'AVB': 2, '3309.T': 1, 'STWD': 1, 'MTO.L': 3, 'KFY': 1, 'BLND.L': 1, 'RGLD': 1, 'KFRC': 1, 'IWG.L': 

,max_sharpe_weight,avg_annual_volatility,avg_annual_return,return_last_period(12m)
RS1.L,0.0,6.298242,64.552049,0.08
KE,0.0,6.424462,22.510101,-0.35
TEG.DE,0.0,5.917304,48.089041,0.63
LEG.DE,0.0,6.162226,66.919311,0.49
SCS,0.0,6.567690,82.924753,0.22
...,...,...,...,...
KVHI,0.0,5.987088,8.968217,-0.12
MOON.L,0.0,5.771122,31.839668,0.22
NEO,0.0,6.763030,53.747281,0.28
6055.T,0.0,7.052233,362.695099,-0.26


In [697]:
mu_6m['KE']

-0.04339569734487636

# Recursive forecasting

## 1 Month

## 6 Months

## 12 Months